## <span style="color:#3A6D8C;">  Download seismic data

#### Wavefroms type: 
- Record: miniseed or other type; 
- station information: stationXML or txt;
- Event information: quakeml or txt
---
#### FDSN rountine web service allow you access any station network data, like iris.

#### orther web service you can use like client.iris or client.earthmorm***


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs

from pathlib import PurePath,Path
from obspy import read, UTCDateTime
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNNoDataException

In [2]:
# check path
client = Client('iris')

# assign data information
start = UTCDateTime("2022-01-01T00:00:00")
end = UTCDateTime("2022-01-03T00:00:00")
network = "IU"
station = "KIP"
location = "00"
channel = "LHZ" # B:Broadband,H: high quality, L: long period,S:short period, Z: vertical component

# get station inventory data
inv = client.get_stations(
    starttime = start,
    endtime = end,
    network = "IU",
    station = station,
    location = location, 
    channel = channel,
)

# save data as daily section
data_list = pd.date_range(start.datetime, end.datetime,freq='D')# unit spacing according by day

# define directionary 
cwd_file = Path().cwd()
for day in data_list:
    str_day = str(day).split(' ')[0]
    filename = f"{network}-{station}-{location}-{channel}-{str_day}"
    
    # check exist
    if any(cwd_file.glob(filename + '.sac')):
        continue;
    else:
        try:
            st = client.get_waveforms(
                network=network,
                station=station,
                location=location,
                channel=channel,
                starttime=UTCDateTime(day),
                endtime=UTCDateTime(day) + 86400, # Only one day's data is taken at a time
            )

            # preprocessing
            pre_filt = (0.005, 0.006, 0.5,0.8)
            tr = st[0].copy()
            tr.detrend('demean')
            tr.detrend('linear')
            tr.taper(0.01)
            tr.remove_response(inventory = inv,output="VEL", water_level=60, pre_filt=pre_filt)
            tr.write(filename + '.sac', format="SAC")
        except FDSNNoDataException:
            print('No waveform in ' + filename)
            continue;
        


In [14]:
inv.plot(projection='global',
         resolution='c',
         color="red",
         label=False,)
plt.show()

## <span style="color:#3A6D8C;"> Station distribution from IU

In [ ]:
client = Client('IRIS')
starttime = UTCDateTime("2018-07-29T00:00:00")
endtime = UTCDateTime("2018-08-02T00:00:00")
network = "IU"
# station = "KIP"
location = "00"
channel = "LHZ"
inv = client.get_stations(
    starttime=starttime,
    endtime=endtime,
    network=network,
    station="*",
    location=location, 
    channel=channel,
    level='response',
)

inv.plot(projection='global',
         resolution='#00758f',
         color="red",
         label=False,)
plt.show()